In [1]:
import pandas as pd 
import numpy as np

**Gaussian Random Projection**

In [2]:
class GaussianRandomProjection:
    def __init__(self, n_components, random_state=None):
        self.n_components = n_components
        self.random_state = random_state
        self.projection_matrix = None

    def fit(self, X):
        n_features = X.shape[1]
        rng = np.random.RandomState(self.random_state)
        
        # Generate a random Gaussian matrix
        self.projection_matrix = rng.normal(0, 1 / np.sqrt(self.n_components), size=(n_features, self.n_components))
        return self

    def transform(self, X):
        if self.projection_matrix is None:
            raise ValueError("Projection matrix not initialized. Call fit first.")
        return X @ self.projection_matrix

    def fit_transform(self, X):
        return self.fit(X).transform(X)

In [3]:
X = np.random.rand(100, 50)
grp = GaussianRandomProjection(n_components=20, random_state=42)
X_projected = grp.fit_transform(X)
print(X_projected.shape)

(100, 20)


**Sparse Random Projection**

In [5]:
class SparseRandomProjection:
    def __init__(self, n_components, density='auto', random_state=None):
        self.n_components = n_components
        self.density = density
        self.random_state = random_state
        self.projection_matrix = None

    def _generate_sparse_matrix(self, n_features):
        rng = np.random.RandomState(self.random_state)

        if self.density == 'auto':
            self.density = 1 / np.sqrt(n_features)  # Recommended in literature

        # Define probability thresholds for sparse matrix
        prob_zero = 1 - 2 * self.density
        prob_pos = self.density
        prob_neg = self.density

        # Generate the random sparse projection matrix
        rand_vals = rng.uniform(0, 1, size=(n_features, self.n_components))
        projection_matrix = np.zeros((n_features, self.n_components))

        projection_matrix[rand_vals < prob_pos] = 1
        projection_matrix[rand_vals > (1 - prob_neg)] = -1

        # Normalize by sqrt(density)
        return projection_matrix / np.sqrt(self.density * n_features)

    def fit(self, X):
        n_features = X.shape[1]
        self.projection_matrix = self._generate_sparse_matrix(n_features)
        return self

    def transform(self, X):
        if self.projection_matrix is None:
            raise ValueError("Projection matrix not initialized. Call fit first.")
        return X @ self.projection_matrix

    def fit_transform(self, X):
        return self.fit(X).transform(X)

In [ ]:
X = np.random.rand(100, 50)  # 100 samples, 50 features
srp = SparseRandomProjection(n_components=20, random_state=42)
X_projected = srp.fit_transform(X)
print(X_projected.shape)  # Should be (100, 20)
X_projected

(100, 20)


**Fast Johnson-Lindenstrauss Lemma**

In [1]:
import numpy as np
from scipy.linalg import hadamard

class FJLTProjection:
    def __init__(self, n_components, random_state=None):
        self.n_components = n_components
        self.random_state = random_state
        self.projection_matrix = None
        self.diagonal_matrix = None
        self.P = None

    def _random_sign_flip(self, n):
        rng = np.random.RandomState(self.random_state)
        return np.diag(rng.choice([-1, 1], size=n))
    
    def _pad_to_power_of_two(self, X):
        n_features = X.shape[1]
        next_power = 2 ** int(np.ceil(np.log2(n_features)))
        if next_power != n_features:
            padding = next_power - n_features
            return np.hstack([X, np.zeros((X.shape[0], padding))])
        return X
    
    def _truncate_back(self, X_transformed, original_dim):
        return X_transformed[:, :original_dim]

    def fit(self, X):
        n_samples, n_features = X.shape
        rng = np.random.RandomState(self.random_state)
        
        # Step 1: Pad input to nearest power of two (Hadamard requires this)
        X_padded = self._pad_to_power_of_two(X)
        padded_dim = X_padded.shape[1]
        
        # Step 2: Create random sign flip matrix D
        self.diagonal_matrix = self._random_sign_flip(padded_dim)
        
        # Step 3: Create random sparse matrix P (column sampling)
        k = self.n_components
        self.P = np.zeros((padded_dim, k))
        selected_cols = rng.choice(padded_dim, size=k, replace=False)
        self.P[selected_cols, np.arange(k)] = 1
        
        # Step 4: Precompute the projection matrix: P^T * H * D
        H = hadamard(padded_dim) / np.sqrt(padded_dim)  # Orthonormal Hadamard
        self.projection_matrix = (self.P.T @ H @ self.diagonal_matrix).T
        
        return self
    
    def transform(self, X):
        if self.projection_matrix is None:
            raise ValueError("Projection matrix not initialized. Call fit first.")
        
        # Pad input to match projection matrix dimensions
        X_padded = self._pad_to_power_of_two(X)
        transformed = X_padded @ self.projection_matrix
        
        # Truncate back if needed (though typically n_components << original dim)
        return transformed
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)

In [2]:
# Generate random data
X = np.random.randn(1000, 784)  # e.g., 1000 samples of 784-dim data

# Apply FJLT
fjlt = FJLTProjection(n_components=100, random_state=42)
X_projected = fjlt.fit_transform(X)

print(X_projected.shape)  # (1000, 100)

(1000, 100)


**Subsample Randomized Hadamard Transform**

In [3]:
import numpy as np
from scipy.linalg import hadamard

class SRHTProjection:
    def __init__(self, n_components, random_state=None):
        self.n_components = n_components
        self.random_state = random_state
        self.projection_matrix = None
        self.diagonal_matrix = None
        self.subsample_indices = None
        self.padded_dim = None

    def _random_sign_flip(self, n):
        rng = np.random.RandomState(self.random_state)
        return np.diag(rng.choice([-1, 1], size=n))
    
    def _pad_to_power_of_two(self, X):
        n_features = X.shape[1]
        next_power = 2 ** int(np.ceil(np.log2(n_features)))
        if next_power != n_features:
            padding = next_power - n_features
            return np.hstack([X, np.zeros((X.shape[0], padding))]), next_power
        return X, n_features

    def fit(self, X):
        rng = np.random.RandomState(self.random_state)
        X_padded, self.padded_dim = self._pad_to_power_of_two(X)
        
        # Step 1: Random sign flipping (diagonal matrix D)
        self.diagonal_matrix = self._random_sign_flip(self.padded_dim)
        
        # Step 2: Random subsampling of rows (we'll do this during transform)
        self.subsample_indices = rng.choice(
            self.padded_dim, 
            size=self.n_components, 
            replace=False
        )
        
        return self
    
    def transform(self, X):
        if self.diagonal_matrix is None or self.subsample_indices is None:
            raise ValueError("Model not fitted. Call fit first.")
        
        X_padded, _ = self._pad_to_power_of_two(X)
        
        # Apply D (random sign flip)
        X_signed = X_padded @ self.diagonal_matrix
        
        # Apply Hadamard transform to each sample (efficient using FHT)
        H = hadamard(self.padded_dim) / np.sqrt(self.padded_dim)
        X_hadamard = X_signed @ H.T  # Equivalent to H @ X_signed.T then transpose back
        
        # Subsample rows
        X_subsampled = X_hadamard[:, self.subsample_indices]
        
        return X_subsampled
    
    def fit_transform(self, X):
        return self.fit(X).transform(X)

    def fast_hadamard_transform(self, x):
        """Recursive implementation of Fast Hadamard Transform (FHT)"""
        n = len(x)
        if n == 1:
            return x
        half = n // 2
        left = self.fast_hadamard_transform(x[:half])
        right = self.fast_hadamard_transform(x[half:])
        return np.concatenate([left + right, left - right]) / np.sqrt(2)

In [12]:
# Generate random data
X = np.abs(np.random.randn(1000, 784))  # 1000 samples of 784-dim data

# Apply SRHT
srht = SRHTProjection(n_components=100, random_state=42)
X_projected = srht.fit_transform(X)

print(X_projected.shape)  # (1000, 100)
X_projected

(1000, 100)


array([[ 0.99781745,  0.38341626, -0.6371556 , ...,  0.4014371 ,
        -0.10208062,  0.35466871],
       [ 1.70090275, -0.07608316, -0.45179103, ..., -0.4812189 ,
         0.7495866 , -0.3840854 ],
       [ 0.11998658, -0.87204892, -0.03830671, ...,  0.03580409,
        -0.27628284, -0.85517851],
       ...,
       [ 1.23364522,  0.58882685, -1.12380621, ...,  0.3836288 ,
        -0.32776447, -0.49034216],
       [ 0.29777458, -0.17806748, -0.26375806, ...,  0.15823629,
        -0.03718125, -0.55514711],
       [ 1.49061235, -0.14056137, -0.0435471 , ...,  0.61098549,
         0.32137928, -1.31358769]])

**Clarkson-Woodruff Transform**

In [6]:
import numpy as np
from scipy import sparse

class ClarksonWoodruffTransform:
    def __init__(self, n_components, random_state=None):
        """
        Parameters:
        -----------
        n_components : int
            Target dimension after projection
        random_state : int or None
            Seed for reproducibility
        """
        self.n_components = n_components
        self.random_state = random_state
        self.projection_matrix = None
        self.n_features = None

    def _generate_sparse_matrix(self, n_rows, n_cols):
        """Generate a sparse CountSketch matrix"""
        rng = np.random.RandomState(self.random_state)
        
        # Random column indices (one per row)
        col_indices = rng.randint(0, n_cols, size=n_rows)
        
        # Random ±1 values
        values = rng.choice([-1, 1], size=n_rows)
        
        # Row indices (0 to n_rows-1)
        row_indices = np.arange(n_rows)
        
        return sparse.csr_matrix(
            (values, (row_indices, col_indices)),
            shape=(n_rows, n_cols)
        )

    def fit(self, X):
        """Generate the sparse projection matrix"""
        self.n_features = X.shape[1]
        
        # Generate sparse CountSketch matrix
        self.projection_matrix = self._generate_sparse_matrix(
            self.n_components,
            self.n_features
        )
        return self

    def transform(self, X):
        """Apply the sparse projection"""
        if self.projection_matrix is None:
            raise ValueError("Projection matrix not initialized. Call fit first.")
        return X @ self.projection_matrix.T

    def fit_transform(self, X):
        """Combined fit and transform"""
        return self.fit(X).transform(X)

In [7]:
# Generate some high-dimensional sparse data
X = sparse.random(1000, 10000, density=0.01, random_state=42)

# Apply Clarkson-Woodruff Transform
cwt = ClarksonWoodruffTransform(n_components=500, random_state=42)
X_projected = cwt.fit_transform(X)

print(f"Original shape: {X.shape}")
print(f"Projected shape: {X_projected.shape}")

Original shape: (1000, 10000)
Projected shape: (1000, 500)


*Compression Algorithms*

In [1]:
import numpy as np
from numpy.fft import fft

def standard_compression(A, r, rOV, w):
    m, n = A.shape
    d = r + rOV  # Effective reduced dimension

    # Step 1: Draw a Gaussian random matrix Omega_L
    Omega_L = np.random.randn(n, d)

    # Step 2: Compute B = (A A^T)^w A Omega_L
    B = A @ Omega_L  # Initial multiplication: A Omega_L
    for _ in range(w):
        B = (A @ A.T) @ B # Power iteration: (A A^T) B

    # Step 3: Compute the orthogonal basis Q using QR decomposition
    Q, _ = np.linalg.qr(B)

    return Q

def srht_compression(A, r, rOV, w):

    m, n = A.shape
    d = r + rOV  # Effective reduced dimension

    # Step 1: Generate subsampled randomized Hadamard matrix Omega_L
    # Find smallest power of 2 greater than or equal to n
    n_power = 2**int(np.ceil(np.log2(n)))
    
    # Generate Hadamard matrix of size n_power x n_power
    H = hadamard(n_power)
    
    # Generate random diagonal matrix D with ±1 entries
    D = np.diag(np.random.choice([-1, 1], size=n_power))
    
    # Compute randomized Hadamard transform
    Omega_full = (1/np.sqrt(n_power)) * D @ H
    
    # If n < n_power, truncate the matrix to size n
    if n < n_power:
        Omega_full = Omega_full[:n, :]
    
    # Randomly subsample d columns
    subsample_indices = np.random.choice(n_power, size=d, replace=False)
    Omega_L = Omega_full[:, subsample_indices]

    # Step 2: Compute B = (A A^T)^w A Omega_L
    B = A @ Omega_L  # Initial multiplication: A Omega_L
    for _ in range(w):
        B = (A @ A.T) @ B  # Power iteration: (A A^T) B

    # Step 3: Compute the orthogonal basis Q using QR decomposition
    Q, _ = np.linalg.qr(B)

    return Q

def fjlt_compression(A, r, rOV, w):
    m, n = A.shape
    d = r + rOV  # Effective reduced dimension

    # Find smallest power of 2 >= n
    n_power = 2**int(np.ceil(np.log2(n)))

    # P: Random diagonal matrix with ±1 entries
    P = np.random.choice([-1, 1], size=n_power)
    
    # D: Sparse random projection matrix
    q = 0.1
    D = (1/np.sqrt(q)) * np.random.choice([0, 1, -1], size=(n_power, d), 
                                        p=[1-q, q/2, q/2])
    
    # H: Fast Walsh-Hadamard Transform via FFT
    def apply_hadamard(x):
        n_pow = len(x)
        # Pad x to n_power if necessary
        if len(x) < n_pow:
            x = np.pad(x, (0, n_pow - len(x)), mode='constant')
        return fft(x) * (1/np.sqrt(n_pow))
    
    # Construct Omega_L = D.T @ H @ P (applied to columns of A)
    # Step 1: Apply P (element-wise multiplication) to each column of A
    if n < n_power:
        A_padded = np.pad(A, ((0, 0), (0, n_power - n)), mode='constant')
    else:
        A_padded = A
    
    PA = A_padded * P[np.newaxis, :]  # (m × n_power)
    
    # Step 2: Apply Hadamard transform to each row of PA
    HPA = np.apply_along_axis(apply_hadamard, 1, PA)  # (m × n_power)
    
    # Step 3: Apply sparse projection D.T
    Omega_L = HPA @ D  # (m × n_power) @ (n_power × d) = (m × d)
    
    # Step 2: Compute B = (A A^T)^w A Omega_L
    B = Omega_L  # Since Omega_L is already m × d, no need for A @ Omega_L here
    for _ in range(w):
        B = (A @ A.T) @ B
    
    # Step 3: QR decomposition
    Q, _ = np.linalg.qr(B)
    return Q

def cwt_compression(A, r, rOV, w):
    m, n = A.shape
    d = r + rOV  # Effective reduced dimension

    # Step 1: Construct Clarkson-Woodruff Transform (CountSketch matrix)
    # Parameters for sparsity (s controls the number of non-zero entries per column)
    s = max(1, int(np.ceil(np.log(n) / np.log(d))))  # Adjust sparsity based on problem size
    
    # Hash functions for column indices and signs
    hash_indices = np.random.randint(0, d, size=n)  # h: [n] -> [d]
    signs = np.random.choice([-1, 1], size=n)       # σ: [n] -> {-1, 1}
    
    # Build sparse sketching matrix S (d × n)
    S = np.zeros((d, n))
    for j in range(n):
        S[hash_indices[j], j] = signs[j]
    
    # Step 2: Compute sketch B = A S^T
    B = A @ S.T  # (m × n) @ (n × d) = (m × d)
    
    # Step 3: Power iteration: B = (A A^T)^w B
    for _ in range(w):
        B = (A @ A.T) @ B
    
    # Step 4: QR decomposition to get orthonormal basis
    Q, _ = np.linalg.qr(B)
    
    return Q

*Randomized NMF*

In [2]:
def randomized_nmf(A, r, r_ov=5, w=1, max_iter=500, tol=1e-6,compression = 'standard'):
    """
    NMF with compression matrices L and R, random initialization for Y_k,
    and multiplicative updates with proper dimension handling.
    """
    m, n = A.shape
    d = r + r_ov  # Effective reduced dimension

    # --- Step 1: Compute compression matrices L and R ---
    if compression == 'standard':
        L = standard_compression(A, r, r_ov, w)      # m × d
        R = standard_compression(A.T, r, r_ov, w).T  # d × n
    elif compression == 'srht':
        L = srht_compression(A, r, r_ov, w)      # m × d
        R = srht_compression(A.T, r, r_ov, w).T  # d × n
    elif compression == 'fjlt':
        L = fjlt_compression(A, r, r_ov, w)      # m × d
        R = fjlt_compression(A.T, r, r_ov, w).T  # d × n
    elif compression == 'cwt':
        L = cwt_compression(A, r, r_ov, w)      # m × d
        R = cwt_compression(A.T, r, r_ov, w).T  # d × n
    else:
        print('Please select compression Algorithm')
        return

    # --- Step 3: Initialize Y_k as random non-negative ---
    Y_k = np.abs(np.random.randn(r, n))  # r × n
    Y_k = np.maximum(Y_k, 1e-6)          # Enforce non-negativity

    # Initialize X
    X_k = np.abs(np.random.randn(m, r))   # m × r
    X_k = np.maximum(X_k, 1e-6)

    # --- Main loop ---
    errors = []
    for k in range(max_iter):
        # --- Update X_{k+1} ---
        # numerator: (m × d) @ (d × r) = m × r
        numerator = (A @ R.T) @ (R @ Y_k.T)
        # denominator: (m × r) @ (r × d) @ (d × r) = m × r
        denominator = X_k @ (Y_k @ R.T @ R @ Y_k.T)
        X_k_plus1 = X_k * numerator / (denominator + 1e-10)
        X_k_plus1 = np.maximum(X_k_plus1, 1e-6)

        # --- Update Y_{k+1} ---
        # numerator: (r × d) @ (d × n) = r × n
        numerator = (L.T @ X_k_plus1).T @ (L.T @ A)
        # denominator: (r × d) @ (d × r) @ (r × n) = r × n
        denominator = (L.T @ X_k_plus1).T @ (L.T @ X_k_plus1) @ Y_k
        Y_k_plus1 = Y_k * numerator / (denominator + 1e-10)
        Y_k_plus1 = np.maximum(Y_k_plus1, 1e-6)

        # --- Convergence check ---
        error = np.linalg.norm(A - X_k_plus1 @ Y_k_plus1, 'fro') / np.linalg.norm(A, 'fro')
        errors.append(error)

        X_k, Y_k = X_k_plus1, Y_k_plus1

    return X_k, Y_k, errors